__Probing Language Models__

This notebook serves as a start for your NLP2 assignment on probing Language Models. This notebook will become part of the contents that you will submit at the end, so make sure to keep your code (somewhat) clean :-)

__note__: This is the first time _anyone_ is doing this assignment. That's exciting! But it might well be the case that certain aspects are too unclear. Do not hesitate at all to reach to me once you get stuck, I'd be grateful to help you out.

__note 2__: This assignment is not dependent on big fancy GPUs. I run all this stuff on my own 3 year old CPU, without any Colab hassle. So it's up to you to decide how you want to run it.

# Models

For the Transformer models you are advised to make use of the `transformers` library of Huggingface: https://github.com/huggingface/transformers
Their library is well documented, and they provide great tools to easily load in pre-trained models.

In [ ]:
from transformers import GPT2Model, GPT2Tokenizer
import torch
print("Imports ready")


In [ ]:
model = GPT2Model.from_pretrained('distilgpt2', output_hidden_states=True)
print("Model ready")
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
print("Tokenizer ready")
# Note that some models don't return the hidden states by default.
# This can be configured by passing `output_hidden_states=True` to the `from_pretrained` method.

In [ ]:
#
## Your code for initializing the rnn model(s)
#
# The Gulordava LSTM model can be found here: 
# https://drive.google.com/open?id=1w47WsZcZzPyBKDn83cMNd0Hb336e-_Sy
#
# N.B: I have altered the RNNModel code to only output the hidden states that you are interested in.
# If you want to do more experiments with this model you could have a look at the original code here:
# https://github.com/facebookresearch/colorlessgreenRNNs/blob/master/src/language_models/model.py
#
from collections import defaultdict
from lstm.model import RNNModel

model_location = 'lstm/gulordava.pt'
lstm = RNNModel('LSTM', 50001, 650, 650, 2)
lstm.load_state_dict(torch.load(model_location))


# This LSTM does not use a Tokenizer like the Transformers, but a Vocab dictionary that maps a token to an id.
with open('lstm/vocab.txt') as f:
    w2i = {w.strip(): i for i, w in enumerate(f)}

vocab = defaultdict(lambda: w2i["<unk>"])
vocab.update(w2i)
i2w = { w2i[k]:k for k in w2i}

In [ ]:
lstm.eval()
print(type(lstm))
with torch.no_grad():
    input_sent = "My sister is 20"
    inputs = torch.tensor([vocab[z] for z in input_sent.split()]).unsqueeze(0)
    outputs = lstm(inputs, lstm.init_hidden(1), True)
    argmaxes = torch.argmax(outputs, dim=1)
    last = argmaxes[-1]
    print("Voc size", outputs.shape[-1])
    print(input_sent, i2w[last.item()])

It is a good idea that before you move on, you try to feed some text to your LMs; and check if everything works accordingly. 

# Data

For this assignment you will train your probes on __treebank__ corpora. A treebank is a corpus that has been *parsed*, and stored in a representation that allows the parse tree to be recovered. Next to a parse tree, treebanks also often contain information about part-of-speech tags, which is exactly what we are after now.

The treebank you will use for now is part of the Universal Dependencies project. I provide a sample of this treebank as well, so you can test your setup on that before moving on to larger amounts of data.

Make sure you accustom yourself to the format that is created by the `conllu` library that parses the treebank files before moving on. For example, make sure you understand how you can access the pos tag of a token, or how to cope with the tree structure that is formed using the `to_tree()` functionality.

In [ ]:
# READ DATA
from typing import List
from conllu import parse_incr, TokenList


# If stuff like `: str` and `-> ..` seems scary, fear not! 
# These are type hints that help you to understand what kind of argument and output is expected.
def parse_corpus(filename: str) -> List[TokenList]:
    data_file = open(filename, encoding="utf-8")

    ud_parses = list(parse_incr(data_file))
    
    return ud_parses


            
            input_text = ''.join(sentence_words)
            input_encoded = tokenizer.encode(input_text)
            with torch.no_grad():
                result = model(torch.tensor(input_encoded).unsqueeze(0))[0]
            result = result.squeeze(0)
            moeilijke_zin = False
            # Now some magic loop
            index = 0
            final_repr = list()
            decoded_repr = [tokenizer.decode(z) for z in input_encoded]
            assert len(result) == len(input_encoded), "Something is terribly wrong!"
            assert len(decoded_repr) == len(input_encoded), "Something is terribly wrong!"
            #print("Treebank:", [z for z in sentence_words])
            #print("Tokenizer:", [tokenizer.decode(z) for z in input_encoded])
            #continue
            for true in sentence_words:
                if index >= len(decoded_repr):
                    # If moeilijke zin: skip deze zin
                    print("Moeilijke zin triggered")
                    moeilijke_zin = True
                    break
                #if len(result)>0: print(index, len(result), "looking for", true)
                # If the words at the position match exactly, just append the representation
                decoded_input_at_position = decoded_repr[index].strip()
                if decoded_input_at_position == true.strip():
                    #print("into real", true)
                    final_repr.append(result[index])
                    index += 1
                    #print("true added whole", true)
                    
                # If the words at the position do not match exactly, search for the end of the token
                # Then average over all the representations in between
                else:
                    #print("into sub", true)
                    offset = 0
                    for search in range(index, len(decoded_repr)):
                        offset += 1
                        true_stripped = true.strip()
                        #print(" ", search, len(sentence_words))
                        current_search = decoded_repr[search].strip()
                        #print("  With sub", current_search)
                        if true_stripped.endswith(current_search):
                            complete_substring = decoded_repr[index:search+1]
                            #print("Substring", true, complete_substring)
                            to_append = torch.mean(result[index:search+1], dim=0)
                            final_repr.append(to_append)
                            #print("true added sub", true)
                            break
                    # Restart search at index + offset
                    index += offset
                    
            # Done
            if not moeilijke_zin:
                final = torch.stack(final_repr)
                final = final.squeeze(0)
                
                if len(final) != len(sentence_words):
                    print(str(sentence_words) + " " + str(decoded_repr))
                if len(final.shape)>1 : 
                    sentences_result.append(final)
                else:
                    sentences_result.append(final.unsqueeze(0))
                for pos_tag in temp_pos_tags: pos_result.append(pos_tag)

# Generating Representations

We now have our data all set, our models are running and we are good to go!

The next step is now to create the model representations for the sentences in our corpora. Once we have generated these representations we can store them, and train additional diagnostic (/probing) classifiers on top of the representations.

There are a few things you should keep in mind here. Read these carefully, as these tips will save you a lot of time in your implementation.
- Transformer models make use of Byte-Pair Encodings (BPE), that chunk up a piece of next in subword pieces. For example, a word such as "largely" could be chunked up into "large" and "ly". We are interested in probing linguistic information on the __word__-level. Therefore, we will follow the suggestion of Hewitt et al. (2019a, footnote 4), and create the representation of a word by averaging over the representations of its subwords. So the representation of "largely" becomes the average of that of "large" and "ly".

- Subword chunks never overlap multiple tokens. In other words, say we have a phrase like "None of the", then the tokenizer might chunk that into "No"+"ne"+" of"+" the", but __not__ into "No"+"ne o"+"f the", as those chunks overlap multiple tokens. This is great for our setup! Otherwise it would have been quite challenging to distribute the representation of a subword over the 2 tokens it belongs to.

- If you closely examine the provided treebank, you will notice that some tokens are split up into multiple pieces, that each have their own POS-tag. For example, in the first sentence the word "Al-Zaman" is split into "Al", "-", and "Zaman". In such cases, the conllu `TokenList` format will add the following attribute: `('misc', OrderedDict([('SpaceAfter', 'No')]))` to these tokens. Your model's tokenizer does not need to adhere to the same tokenization. E.g., "Al-Zaman" could be split into "Al-"+"Za"+"man", making it hard to match the representations with their correct pos-tag. Therefore I recommend you to not tokenize your entire sentence at once, but to do this based on the chunking of the treebank. Make sure to still incoporate the spaces in a sentence though, as these are part of the BPE of the tokenizer. The tokenizer for GPT-2 adds spaces at the start of a token.

- The LSTM LM does not have the issues related to subwords, but is far more restricted in its vocabulary. Make sure you keep the above points in mind though, when creating the LSTM representations. You might want to write separate functions for the LSTM, but that is up to you.

I would like to stress that if you feel hindered in any way by the simple code structure that is presented here, you are free to modify it :-) Just make sure it is clear to an outsider what you're doing, some helpful comments never hurt.

In [ ]:
pos_w2i = dict()
pos_i2w = dict()
last = 0

In [ ]:
from lstm.model import RNNModel


def fetch_sen_reps(ud_parses: List[TokenList], model, tokenizer) -> List[Tensor]:
    """
    Returns a list of length len(ud_parses)
    """
    global last, pos_w2i, pos_i2w
    doing_lstm = type(model) == RNNModel
    sentences_result = list()
    pos_result = list()
    
    
    for sentence_nr, sentence in enumerate(ud_parses):
        temp_pos_tags = list()
        sentence_words = list()
        
        # First build string sentence repr with spaces and such
        for token in sentence:
            #print(token)
            postag = token['upostag']
            if postag in pos_w2i:
                posindex = pos_w2i[postag]
            else:
                posindex = last
                pos_w2i[postag] = last 
                pos_i2w[last] = postag 
                last += 1
            temp_pos_tags.append(posindex)
            if token['misc'] is not None:
                # SpaceAfter = False
                next_word = token['form']
                sentence_words.append(next_word)
            else:
                # SpaceAfter = True
                next_word = token['form'] + ' '
                sentence_words.append(next_word)
            
        # Now build model representation!
        
        # In case of LSTM
        if doing_lstm:
            the_input = torch.tensor([tokenizer[z.strip()] for z in sentence_words]).unsqueeze(0)
            with torch.no_grad():
                final = model(the_input, lstm.init_hidden(1))
            final = final.squeeze(0)
            assert len(final) == len(sentence_words), "Something is wrong.."
            sentences_result.append(final)
            for pos_tag in temp_pos_tags: pos_result.append(pos_tag)
        # In case of Transformer
        else:
            sentence_words_spaces = list()
            for i,word in enumerate(sentence_words):
                if i>0 and sentence_words[i-1][-1] == ' ':
                    sentence_words_spaces.append(' ' + word.strip())
                else:
                    sentence_words_spaces.append(word.strip())
            # Flatten
            rep = [item for sublist in [tokenizer.encode(z) for z in sentence_words_spaces] for item in sublist]
            the_input = torch.tensor(rep)
            with torch.no_grad():
                result = model(the_input)[0]
            decoded_repr = [tokenizer.decode(z) for z in rep]
            final_repr = list()
            index = 0
            # Take means where necessary
            for true in sentence_words:
                true_stripped = true.strip()
                decoded_at_pos = decoded_repr[index].strip()
                # If words are equal, search no further
                if decoded_at_pos == true_stripped:
                    final_repr.append(result[index].unsqueeze(0))
                    index += 1
                # If not equal, search where the current word ends
                # In some papers they simply take the first part of the word only
                # Maybe we can experiment with this as well, as mean/max/first representation can differ a lot
                else:
                    offset=0
                    for search in range(index, len(decoded_repr)):
                        offset += 1
                        current_search = decoded_repr[search].strip()
                        if true_stripped.endswith(current_search):
                            complete_substring = decoded_repr[index:search+1]
                            to_append = torch.mean(result[index:search+1], dim=0)
                            final_repr.append(to_append.unsqueeze(0))
                            break
                    index += offset
                
            assert len(final_repr) == len(sentence_words)
            sentences_result.append(torch.stack(final_repr).squeeze(1))
            for pos_tag in temp_pos_tags: pos_result.append(pos_tag)
        
    #print("Lenyes", len(sentences_result))
    #print("Lengths", [z.shape for z in sentences_result])
    #print("Lenpos", len(pos_result))
    yes = torch.cat([s for s in sentences_result], dim=0)
    pos = torch.tensor(pos_result)
    
    #print("Data shape" ,yes.shape, pos.shape)
    return yes, pos
    
assert_sen_reps(model, tokenizer, lstm, vocab)

In [ ]:
# FETCH SENTENCE REPRESENTATIONS
from torch import Tensor
import pickle


# I provide the following sanity check, that compares your representations against a pickled version of mine.
# Note that I use the DistilGPT-2 LM here. For the LSTM I used 0-valued initial states.
def assert_sen_reps(model, tokenizer, lstm, vocab):
    with open('distilgpt2_emb1.pickle', 'rb') as f:
        distilgpt2_emb1 = pickle.load(f)

    with open('lstm_emb1.pickle', 'rb') as f:
        lstm_emb1 = pickle.load(f)
    
    corpus = parse_corpus('data/sample/en_ewt-ud-train.conllu')[:1]
    #print(len(corpus))
    #corpus = corpus[:50]

    own_distilgpt2_emb1 = fetch_sen_reps(corpus, model, tokenizer)[0]
    own_lstm_emb1 = fetch_sen_reps(corpus, lstm, vocab)[0]
    
    assert distilgpt2_emb1.shape == own_distilgpt2_emb1.shape
    assert lstm_emb1.shape == own_lstm_emb1.shape
    
    assert torch.allclose(distilgpt2_emb1, own_distilgpt2_emb1,atol=1e-05), "GPT2 embeddings don't match!"
    assert torch.allclose(lstm_emb1, own_lstm_emb1,atol=1e-05), "LSTM embeddings don't match!"

    print("All is well!")
    debug = False 
    
    if debug:
        for i in range(len(lstm_emb1)):
            are_all_close = torch.allclose(lstm_emb1[i], own_lstm_emb1[i],atol=1e-05)
            print(i, are_all_close, torch.max(lstm_emb1[i] - own_lstm_emb1[i]))
            if not are_all_close:
                print("LSTM embs don't match as position " + str(i) + " " + corpus[0][i]['form'])

        for i in range(len(lstm_emb1)):
            are_all_close = torch.allclose(distilgpt2_emb1[i], own_distilgpt2_emb1[i] ,atol=1e-05)
            print(i, are_all_close, torch.max(distilgpt2_emb1[i] - own_distilgpt2_emb1[i])) 
            if not are_all_close:
                print("DistilGPT-2 embs don't match at position " + str(i) + " " + corpus[0][i]['form'])


In [ ]:
import os

# Function that combines the previous functions, and creates 2 tensors for a .conllu file: 
# 1 containing the token representations, and 1 containing the (tokenized) pos_tags.

def create_data(filename: str, lm, w2i, pos_vocab=None):
    global pos_w2i
    ud_parses = parse_corpus(filename)
    
    sen_reps, pos_tags = fetch_sen_reps(ud_parses, lm, w2i)
    pos_vocab = pos_w2i
    #pos_tags, pos_vocab = fetch_pos_tags(ud_parses, pos_vocab=pos_vocab)
    
    return sen_reps, pos_tags, pos_vocab


lm = model  # or `lstm`
current_w2i = tokenizer  # or `vocab`
use_sample = True

train_x, train_y, train_vocab = create_data(
    os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-train.conllu'),
    lm, 
    current_w2i
)

dev_x, dev_y, _ = create_data(
    os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-dev.conllu'),
    lm, 
    current_w2i,
    pos_vocab=train_vocab
)

test_x, test_y, _ = create_data(
    os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-dev.conllu'),
    lm,
    current_w2i,
    pos_vocab=train_vocab
)


# Diagnostic Classification

We now have our models, our data, _and_ our representations all set! Hurray, well done. We can finally move onto the cool stuff, i.e. training the diagnostic classifiers (DCs).

DCs are simple in their complexity on purpose. To read more about why this is the case you could already have a look at the "Designing and Interpreting Probes with Control Tasks" by Hewitt and Liang (esp. Sec. 3.2).

A simple linear classifier will suffice for now, don't bother with adding fancy non-linearities to it.

I am personally a fan of the `skorch` library, that provides `sklearn`-like functionalities for training `torch` models, but you are free to train your dc using whatever method you prefer.

As this is an Artificial Intelligence master and you have all done ML1 + DL, I expect you to use your train/dev/test splits correctly ;-)

In [ ]:
import torch.utils.data as data

class TinyDataset(data.Dataset):
    def __init__(self, baby, b):
        self.baby = baby 
        self.b = b
        
    def __len__(self):
        return len(self.baby)

    def __getitem__(self, index):
        return self.baby[index], self.b[index]
    
train_loader = data.DataLoader(TinyDataset(train_x, train_y), batch_size=16)
test_loader = data.DataLoader(TinyDataset(test_x, test_y), batch_size=16)

In [ ]:
# DIAGNOSTIC CLASSIFIER
import torch.nn as nn

for i in range(100):
    ce = nn.CrossEntropyLoss()
    my_model = nn.Linear(768, len(pos_w2i))
    optim = torch.optim.Adam(my_model.parameters())
    for i in range(10):
        for x,y in train_loader:
            outputs = my_model(x)
            preds = torch.argmax(outputs,dim=1)
            correct = torch.sum(torch.eq(preds, y))
            accuracy = correct.item()/y.shape[0]
            loss = ce(outputs, y)

            optim.zero_grad()
            loss.backward()
            optim.step()


    correct = 0.0
    total = 0.0
    for x,y in test_loader:
        outputs = my_model(x)
        preds = torch.argmax(outputs,dim=1)
        c = torch.sum(torch.eq(preds, y))
        correct += c.item()
        total += y.shape[0]

    print(correct/total)

# Trees

For our gold labels, we need to recover the node distances from our parse tree. For this we will use the functionality provided by `ete3`, that allows us to compute that directly. I have provided code that transforms a `TokenTree` to a `Tree` in `ete3` format.

In [ ]:
# In case you want to transform your conllu tree to an nltk.Tree, for better visualisation

def rec_tokentree_to_nltk(tokentree):
    token = tokentree.token["form"]
    tree_str = f"({token} {' '.join(rec_tokentree_to_nltk(t) for t in tokentree.children)})"

    return tree_str


def tokentree_to_nltk(tokentree):
    from nltk import Tree as NLTKTree

    tree_str = rec_tokentree_to_nltk(tokentree)

    return NLTKTree.fromstring(tree_str)

In [ ]:
# !pip install ete3
from ete3 import Tree as EteTree


class FancyTree(EteTree):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, format=1, **kwargs)
        
    def __str__(self):
        return self.get_ascii(show_internal=True)
    
    def __repr__(self):
        return str(self)


def rec_tokentree_to_ete(tokentree):
    idx = str(tokentree.token["id"])
    children = tokentree.children
    if children:
        return f"({','.join(rec_tokentree_to_ete(t) for t in children)}){idx}"
    else:
        return idx
    
def tokentree_to_ete(tokentree):
    newick_str = rec_tokentree_to_ete(tokentree)

    return FancyTree(f"{newick_str};")

In [ ]:
# Let's check if it works!
# We can read in a corpus using the code that was already provided, and convert it to an ete3 Tree.

def parse_corpus(filename):
    from conllu import parse_incr

    data_file = open(filename, encoding="utf-8")

    ud_parses = list(parse_incr(data_file))
    
    return ud_parses

corpus = parse_corpus('data/sample/en_ewt-ud-train.conllu')
item = corpus[0]
tokentree = item.to_tree()
ete3_tree = tokentree_to_ete(tokentree)
print(ete3_tree)

As you can see we label a token by its token id (converted to a string). Based on these id's we are going to retrieve the node distances.

To create the true distances of a parse tree in our treebank, we are going to use the `.get_distance` method that is provided by `ete3`: http://etetoolkit.org/docs/latest/tutorial/tutorial_trees.html#working-with-branch-distances

We will store all these distances in a `torch.Tensor`.

Please fill in the gap in the following method. I recommend you to have a good look at Hewitt's blog post  about these node distances.

In [ ]:
def create_gold_distances(corpus):
    all_distances = []

    for item in (corpus):
        tokentree = item.to_tree()
        ete_tree = tokentree_to_ete(tokentree)

        sen_len = len(ete_tree.search_nodes())
        distances = torch.zeros((sen_len, sen_len))

        # Your code for computing all the distances comes here.

        all_distances.append(distances)

    return all_distances

The next step is now to do the previous step the other way around. After all, we are mainly interested in predicting the node distances of a sentence, in order to recreate the corresponding parse tree.

Hewitt et al. reconstruct a parse tree based on a _minimum spanning tree_ (MST, https://en.wikipedia.org/wiki/Minimum_spanning_tree). Fortunately for us, we can simply import a method from `scipy` that retrieves this MST.

In [ ]:
from scipy.sparse.csgraph import minimum_spanning_tree
import torch


def create_mst(distances):
    distances = torch.triu(distances).detach().numpy()
    
    mst = minimum_spanning_tree(distances).toarray()
    mst[mst>0] = 1.
    
    return mst

Let's have a look at what this looks like, by looking at a relatively short sentence in the sample corpus.

If your addition to the `create_gold_distances` method has been correct, you should be able to run the following snippet. This then shows you the original parse tree, the distances between the nodes, and the MST that is retrieved from these distances. Can you spot the edges in the MST matrix that correspond to the edges in the parse tree?

In [ ]:
item = corpus[5]
tokentree = item.to_tree()
ete3_tree = tokentree_to_ete(tokentree)
print(ete3_tree, '\n')

gold_distance = create_gold_distances(corpus[5:6])[0]
print(gold_distance, '\n')

mst = create_mst(gold_distance)
print(mst)

Now that we are able to map edge distances back to parse trees, we can create code for our quantitative evaluation. For this we will use the Undirected Unlabeled Attachment Score (UUAS), which is expressed as:

$$\frac{\text{number of predicted edges that are an edge in the gold parse tree}}{\text{number of edges in the gold parse tree}}$$

To do this, we will need to obtain all the edges from our MST matrix. Note that, since we are using undirected trees, that an edge can be expressed in 2 ways: an edge between node $i$ and node $j$ is denoted by both `mst[i,j] = 1`, or `mst[j,i] = 1`.

You will write code that computes the UUAS score for a matrix of predicted distances, and the corresponding gold distances. I recommend you to split this up into 2 methods: 1 that retrieves the edges that are present in an MST matrix, and one general method that computes the UUAS score.

In [ ]:
def edges(mst):
    edges = set()

    # Your code for retrieving the edges from the MST matrix

    return edges

def calc_uuas(pred_distances, gold_distances):
    uuas = None
    
    # Your code for computing the UUAS score
    
    return uuas


# Structural Probes

We now have everything in place to start doing the actual exciting stuff: training our structural probe!
    
To make life easier for you, we will simply take the `torch` code for this probe from John Hewitt's repository. This allows you to focus on the training regime from now on.

In [ ]:
import torch.nn as nn
import torch


class StructuralProbe(nn.Module):
    """ Computes squared L2 distance after projection by a matrix.
    For a batch of sentences, computes all n^2 pairs of distances
    for each sentence in the batch.
    """
    def __init__(self, model_dim, rank, device="cpu"):
        super().__init__()
        self.probe_rank = rank
        self.model_dim = model_dim
        
        self.proj = nn.Parameter(data = torch.zeros(self.model_dim, self.probe_rank))
        
        nn.init.uniform_(self.proj, -0.05, 0.05)
        self.to(device)

    def forward(self, batch):
        """ Computes all n^2 pairs of distances after projection
        for each sentence in a batch.
        Note that due to padding, some distances will be non-zero for pads.
        Computes (B(h_i-h_j))^T(B(h_i-h_j)) for all i,j
        Args:
          batch: a batch of word representations of the shape
            (batch_size, max_seq_len, representation_dim)
        Returns:
          A tensor of distances of shape (batch_size, max_seq_len, max_seq_len)
        """
        transformed = torch.matmul(batch, self.proj)
        
        batchlen, seqlen, rank = transformed.size()
        
        transformed = transformed.unsqueeze(2)
        transformed = transformed.expand(-1, -1, seqlen, -1)
        transposed = transformed.transpose(1,2)
        
        diffs = transformed - transposed
        
        squared_diffs = diffs.pow(2)
        squared_distances = torch.sum(squared_diffs, -1)

        return squared_distances

    
class L1DistanceLoss(nn.Module):
    """Custom L1 loss for distance matrices."""
    def __init__(self):
        super().__init__()

    def forward(self, predictions, label_batch, length_batch):
        """ Computes L1 loss on distance matrices.
        Ignores all entries where label_batch=-1
        Normalizes first within sentences (by dividing by the square of the sentence length)
        and then across the batch.
        Args:
          predictions: A pytorch batch of predicted distances
          label_batch: A pytorch batch of true distances
          length_batch: A pytorch batch of sentence lengths
        Returns:
          A tuple of:
            batch_loss: average loss in the batch
            total_sents: number of sentences in the batch
        """
        labels_1s = (label_batch != -1).float()
        predictions_masked = predictions * labels_1s
        labels_masked = label_batch * labels_1s
        total_sents = torch.sum((length_batch != 0)).float()
        squared_lengths = length_batch.pow(2).float()

        if total_sents > 0:
            loss_per_sent = torch.sum(torch.abs(predictions_masked - labels_masked), dim=(1,2))
            normalized_loss_per_sent = loss_per_sent / squared_lengths
            batch_loss = torch.sum(normalized_loss_per_sent) / total_sents
        
        else:
            batch_loss = torch.tensor(0.0)
        
        return batch_loss, total_sents


I have provided a rough outline for the training regime that you can use. Note that the hyper parameters that I provide here only serve as an indication, but should be (briefly) explored by yourself.

As can be seen in Hewitt's code above, there exists functionality in the probe to deal with batched input. It is up to you to use that: a (less efficient) method can still incorporate batches by doing multiple forward passes for a batch and computing the backward pass only once for the summed losses of all these forward passes. (_I know, this is not the way to go, but in the interest of time that is allowed ;-), the purpose of the assignment is writing a good paper after all_).

In [ ]:
from torch import optim

'''
Similar to the `create_data` method of the previous notebook, I recommend you to use a method 
that initialises all the data of a corpus. Note that for your embeddings you can use the 
`fetch_sen_reps` method again. However, for the POS probe you concatenated all these representations into 
1 big tensor of shape (num_tokens_in_corpus, model_dim). 

The StructuralProbe expects its input to contain all the representations of 1 sentence, so I recommend you
to update your `fetch_sen_reps` method in a way that it is easy to retrieve all the representations that 
correspond to a single sentence.
''' 

def init_corpus(path, concat=False, cutoff=None):
    """ Initialises the data of a corpus.
    
    Parameters
    ----------
    path : str
        Path to corpus location
    concat : bool, optional
        Optional toggle to concatenate all the tensors
        returned by `fetch_sen_reps`.
    cutoff : int, optional
        Optional integer to "cutoff" the data in the corpus.
        This allows only a subset to be used, alleviating 
        memory usage.
    """
    corpus = parse_corpus(path)[:cutoff]

    embs = fetch_sen_reps(corpus, model, tokenizer, concat=concat)    
    gold_distances = create_gold_distances(corpus)
    
    return gold_distances, embs


# I recommend you to write a method that can evaluate the UUAS & loss score for the dev (& test) corpus.
# Feel free to alter the signature of this method.
def evaluate_probe(probe, _data):
    # YOUR CODE HERE
    
    return loss_score, uuas_score


# Feel free to alter the signature of this method.
def train(_data):
    emb_dim = 768
    rank = 64
    lr = 10e-4
    batch_size = 24

    probe = StructuralProbe(emb_dim, rank)
    optimizer = optim.Adam(probe.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5,patience=1)
    loss_function =  L1DistanceLoss()

    for epoch in range(epochs):

        for i in range(0, len(corpus), batch_size):
            optimizer.zero_grad()

            # YOUR CODE FOR DOING A PROBE FORWARD PASS

            batch_loss.backward()
            optimizer.step()

        dev_loss, dev_uuas = evaluate_probe(probe, _dev_data)

        # Using a scheduler is up to you, and might require some hyper param fine-tuning
        scheduler.step(dev_loss)

    test_loss, test_uuas = evaluate_probe(probe, _test_data)